In [22]:
import cv2
import numpy as np
from skimage.draw import line

In [ ]:
num_iterations = 10000

In [24]:
def disp(canvas = canvas):
    cv2.imshow('Circles', canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [25]:
def generate_points(cx, cy, radius, num_points):
    """Generate points evenly spaced on a circle with center (cx, cy) and given radius."""
    points = []
    for i in range(num_points):
        theta = 2 * np.pi * i / num_points
        x = cx + radius * np.cos(theta)
        y = cy + radius * np.sin(theta)
        points.append((int(round(x)), int(round(y))))
    return points

In [26]:
h = 1000
w = 1000
canvas = np.full((h, w, 3), 255, dtype=np.uint8)
thickness = 1
points = generate_points(w/2,h/2,w/2,500)
for center_x, center_y in points:
    cv2.circle(canvas, (center_x, center_y), 1, 0 , thickness)
disp()


In [27]:
def get_line_pixels(p1, p2):
    """Get the list of pixels along a line between two points."""
    x1, y1 = p1
    x2, y2 = p2
    rr, cc = line(y1, x1, y2, x2)
    return list(zip(cc, rr))  # Returns list of (x,y) coordinates

In [28]:
lin = get_line_pixels(points[0],points[100])
for center_x, center_y in lin:
    cv2.circle(canvas, (center_x, center_y), 1, 0 , thickness)
disp()

In [71]:
image_path = 'circle2.png'  # Replace with your image path
original_image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
height, width = original_image.shape
minVal, maxVal = 100, 200
edges = cv2.Canny(original_image, minVal, maxVal)
img = cv2.bitwise_not(edges)
disp(img)